# Evaluation - stage cartographie de l'écosystème

Ce notebook est à réaliser chez soi dans un délai de 2 jours, mais rassurez-vous, ça ne prend pas autant de temps ! Il a pour objectif d'évaluer certaines de vos capacités en code et vos bonnes pratiques en scraping, NLP (Natural Language Processing) et manipulation d'API.

L'objectif de ce notebook n'est pas de produire des outils opérationnels, mais d'évaluer vos compétences ou votre capacité à chercher des informations.

En cas de doute ou de question, n'hésitez pas à nous écrire.

# 0. Git

La première chose à faire est de "récupérer" ce notebook pour le compléter. Plusieurs options selon votre maîtrise de Git et/ou que vous ayez un compte :
- fork le repo suivant et le compléter sur un projet perso
- vous ajouter au projet, créez votre branch `submission_nomprenom` et push dessus
- vous envoyez par mail le notebook à compléter et renvoyer par mail

https://github.com/TitouanBlaize/carto_affi/blob/main/evaluation_git.ipynb

## 1. NLP

Cette première partie évalue vos connaissances des packages de NLP et vos bonnes pratiques quant au preprocessing de texte.

In [388]:
# écrivez ici une liste de librairies que vous connaissez pour faire du NLP
#import nltk
#import spacy
#import gensim
#import textblob
#import torchtext

Dans une première sous-partie, nous vous proposons de coder un pipeline/fonction de preprocessing de données textuelles. 

L'objectif est, à partir de textes bruts, de produire des textes propres (ponctuation, casse, pluriel, enlever les stopwords,...) qui peuvent être utilisés en entrée d'un pipeline d'apprentissage/fonction. 
Libre à vous d'inclure et d'implémenter toutes les fonctions de nettoyage que vous souhaitez.
Ces opérations peuvent être menées au sein d'une classe (cf class NLPPipeline) ou d'une fonction (NLP_cleaning), comme vous préférez.

In [389]:
# Via une classe ("Orienté objet")

#importation des librairies nécessaires
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

class NLPPipeline():
    def __init__(self):
        nltk.download('punkt')
        nltk.download('wordnet')
        nltk.download('stopwords')
        self.lemmatizer = WordNetLemmatizer() #Récupération du lemmatiseur
        self.stop_words = set(stopwords.words('french')) #Récupération des stopwords

    def transform(self, x):

        #Transformation du texte en minuscule
        textLower = x.lower()

        #Suppression des chiffres ainsi que des signes de ponctuation
        textCleaned = ""
        for c in textLower:
            if not c.isdigit() and c not in string.punctuation:
                textCleaned += c

        #Tokenization
        tokens = word_tokenize(textCleaned, language='french')

        #Lemmatization
        lemmatizedTokens = []
        for token in tokens:
            lemmatizedTokens.append(self.lemmatizer.lemmatize(token))

        #Suppression des stopwords
        filteredTokens = []
        for token in lemmatizedTokens:
            if token not in self.stop_words:
                filteredTokens.append(token)

        #Reconstruction du texte
        cleanedText = ' '.join(filteredTokens)

        return cleanedText

In [390]:
# Via une fonction

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Télécharger les données nécessaires de nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def NLP_cleaning(x):
    lemmatizer = WordNetLemmatizer() #lemmatiseur
    stop_words = set(stopwords.words('french')) # stopwords

    #Transformation du texte en minuscule
    text_lower = x.lower()

    #Suppression de la ponctuation et des chiffres
    text_cleaned = ""
    for c in text_lower:
        if not c.isdigit() and c not in string.punctuation:
            text_cleaned += c

    #Tokenization
    tokens = word_tokenize(text_cleaned, language='french')

    #Lemmatization
    lemmatized_tokens = []
    for token in tokens:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))

    #Suppression stopwords
    filtered_tokens = []
    for token in lemmatized_tokens:
        if token not in stop_words:
            filtered_tokens.append(token)

    #Reconstruction du texte
    cleaned_text = ' '.join(filtered_tokens)

    return cleaned_text


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [391]:
pipeline = NLPPipeline()
pipeline.transform("Georges Washington, 1er président des USA, habite à Washington.")
NLP_cleaning("Georges Washington, 1er président des USA, habite à Washington.")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nawalbendjelloul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'george washington er président usa habite washington'

On pourra par exemple essayer d'obtenir `georges washington er president usa habite a washington`, il n'y a pas de "bon" résultat, libre à vous d'ajouter plusieurs cleaning de textes.

## 2. Scraping/API

Dans cette deuxième partie, nous vous proposons d'écrire une petite fonction de scraping.

A l'aide de la librairie de votre choix, écrivez une fonction qui permet de trouver le nombre de citations d'un article sur Pubmed.   
Vous appliquerez la fonction à l'article suivant: "New onset refractory status epilepticus (NORSE) *C. Sculier, N. Gaspard*, 2019" dont l'ID Pubmed est 30482654

https://pubmed.ncbi.nlm.nih.gov/30482654/

In [392]:
from bs4 import BeautifulSoup
import requests

In [393]:
import requests

def get_number_of_citations(pubmed_id):
    #Création l'URL de l'API Europe PMC  avec l'ID Pubmed
    url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=EXT_ID:{pubmed_id}&format=json'

    # Envoi d'une requête GET à l'URL
    response = requests.get(url)

    # Vérification de l'exécution correcte de la requête
    if response.status_code == 200:

        # Extraction des données JSON de la réponse
        data = response.json()

        # Vérification de l'envoi des réponses
        if data.get("resultList") and data["resultList"].get("result"):

            # Extraction du nombre de citations du premier résultat
            citation_count = data["resultList"]["result"][0].get("citedByCount")

            # Vérification de l'existence du nombre de citations
            if citation_count is not None:
                return int(citation_count)
            else:
                return "Citation count not found."
        else:
            return "Article not found."
    else:
        return f"Error {response.status_code}: Could not fetch the URL."

En runnant la case suivante, on est censé obtenir 19, bonne chance !

In [394]:
print(get_number_of_citations(30482654))

17
